<a href="https://colab.research.google.com/github/durml91/Personal/blob/main/Transformer_implementation_I.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install einops
!pip install equinox

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.6/125.6 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for jax: filename=jax-0.4.13-py3-none-any.whl size=1518707 sha256=a4bc967e75216f38bbf04110120a0b43e297237bbea9caed108d62ce06c7b3ef
  Stored in directory: /root/.cache/pip/wheels/f3/7a/25/f297f69029b5e4064e4736a0c4b3996a44cc27781c120bcb99
Successfully built jax
  Attempting uninstall: jax
    Found existing installation: jax 0.4.10
    Uninstalling jax-0.4.10:
      Successfully uninstalled jax-0.4.10


In [2]:
pip install -U jax jaxlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 MB 7.8 MB/s eta 0:00:00
  Attempting uninstall: jaxlib
    Found existing installation: jaxlib 0.4.10+cuda11.cudnn86
    Uninstalling jaxlib-0.4.10+cuda11.cudnn86:
      Successfully uninstalled jaxlib-0.4.10+cuda11.cudnn86


In [3]:
import jax
import jax.random as jr
import jax.numpy as jnp
import einops
import equinox as eqx
import optax
import tqdm

In [4]:
key = jr.PRNGKey(2022)

### Transformer modules

Embedding table

In [7]:
class InputEmbeddings(eqx.Module):

  embedding: eqx.nn.Embedding

  def __init__(self, d_model: int, vocab_size: int, key):

      self.d_model = d_model
      self.vocab_size = vocab_size
      self.embedding = eqx.nn.Embedding(vocab_size, d_model, key)

  def __call__(self, x):
    return self.embedding(x)* jnp.sqrt(self.d_model)

Positional encoding

In [10]:
class PositionalEncoding(eqx.Module):

    dropout: eqx.nn.Dropout

    def __init__(self, d_model: int, seq_len: int, dropout_rate:float, key) -> None:

        self.d_model = d_model
        self.seq_len = seq_len
        self.dropout = eqx.nn.Dropout(dropout_rate)

        pe = jnp.zeros((seq_len, d_model), dtype = float)

        position = jnp.arange(0, seq_len, dtype=float)
        position = jnp.expand_dims(position, 1)

        div_term = jnp.exp(jnp.arange(0, d_model, 2) * (-jnp.log(10000.0) / d_model))

        pe[:, 0::2] = jnp.sin(position * div_term)
        pe[:, 1::2] = jnp.cos(position * div_term)

        pe = jnp.expand_dims(pe, 0)

    def __call__(self, x):

        x = x + (self.pe[:, :x.shape[1], :])

        return self.dropout(x)


####### need to freeze parameters

In [ ]:
class LayerNormalisation(eqx.Module):

    def __init__(self, eps: float)